比賽名稱 : House Prices - Advanced Regression Techniques。
動機 : 會選擇這個競賽最主要的原因是主題和課程內容學到的有相關，起初我本來以為這個主題和上課教的案例一樣，後來深入去了解它的資料集後，發現比上課所學還要複雜很多，也覺得可以藉由這個競賽來複習並加強上課學到的觀念。
介紹 : 資料集的部分包含了80個不同的欄位，和課堂範例相比難度提升許多，從房子的周遭像是街道巷弄、地板材質，到房子內部幾房幾廳、裝潢及建材材質等。目標是藉由資料集給的欄位來預測房價

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb

dataset=pd.read_csv("train.csv")
'''x=dataset.iloc[1:81]
for i in x:
    sb.jointplot(data=dataset,x=i,y="SalePrice")
'''
#sb.jointplot(data=dataset,x="Street",y="SalePrice")
#sb.jointplot(data=dataset,x="Alley",y="SalePrice")
#sb.jointplot(data=dataset,x="Utilities",y="SalePrice")
#sb.jointplot(data=dataset,x="PoolArea",y="SalePrice")
dataset.drop(["Street","Alley","Utilities","PoolArea","PoolQC","Fence","MiscFeature","MiscVal"],axis=1,inplace=True)
dataset.head()
dataset.info()


dataset["LotFrontage"].mean()
dataset["LotFrontage"]=dataset["LotFrontage"].fillna(dataset["LotFrontage"].mean())


dataset["MasVnrType"].value_counts().idxmax()
dataset["MasVnrType"].fillna(dataset["MasVnrType"].value_counts().idxmax(),inplace=True)


dataset["MasVnrArea"].value_counts().idxmax()
dataset["MasVnrArea"].fillna(dataset["MasVnrArea"].value_counts().idxmax(),inplace=True)

dataset["BsmtQual"].fillna("No",inplace=True)
dataset["BsmtCond"].fillna("No",inplace=True)
dataset["BsmtExposure"].fillna("Nb",inplace=True)
dataset["BsmtFinType1"].fillna("Nb",inplace=True)
dataset["BsmtFinType2"].fillna("Nb",inplace=True)

dataset["Electrical"].value_counts().idxmax()
dataset["Electrical"].fillna(dataset["Electrical"].value_counts().idxmax(),inplace=True)

dataset["FireplaceQu"].fillna("Nf",inplace=True)
dataset["GarageType"].fillna("Ng",inplace=True)
dataset["GarageYrBlt"].fillna(0,inplace=True)
dataset["GarageFinish"].fillna("Ng",inplace=True)
dataset["GarageQual"].fillna("Ng",inplace=True)
dataset["GarageCond"].fillna("Ng",inplace=True)

miss=dataset.isnull().sum()!=1460
print(dataset.isnull().sum().to_string())

ds=pd.get_dummies(data=dataset)
ds.head()
ds.info()
ds.corr()
x=ds.drop(["SalePrice"],axis=1)
y=ds["SalePrice"]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=54)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)
prediction=lr.predict(x_test)
prediction

import joblib
joblib.dump(lr,"AdvanceHousePricing.pkl",compress=3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 73 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   LotConfig      1460 non-null   object 
 8   LandSlope      1460 non-null   object 
 9   Neighborhood   1460 non-null   object 
 10  Condition1     1460 non-null   object 
 11  Condition2     1460 non-null   object 
 12  BldgType       1460 non-null   object 
 13  HouseStyle     1460 non-null   object 
 14  OverallQual    1460 non-null   int64  
 15  OverallCond    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

['AdvanceHousePricing.pkl']

資料處理 :
    藉由觀察kaggle上資料集的統整，在Street欄位中發現只有少數幾筆為Grvl，推測該欄位不影響結果
    而在Alley欄位超過90%的值為NA且經由圖推測該欄位對於結果應該也沒有影響
    Utilities的部分超過99%一樣，同樣推測該欄位不影響結果
    PoolArea和PoolQC的部分，剛開始我認為有Pool的房價一定會很高，但透過圖發現Pool對於房價沒有決定性的影響，我認為可能原因是樣本數太少，資料集提供的房子有Pool的數量很少
    另外如Fence、MiscFeature及MiscVal欄位也是因為缺值過多所以忽略

缺值處理 :
    大部分的欄位會有缺值並不是因為真的沒有記錄到，而是沒有那個屬性，以GarageType為例，NA表示的是該房子沒有車庫，針對這種類型的缺值處理，我是利用給予那些缺值一個新的屬性，這樣一來既可以處理缺值的問題又可以保證預測結果不會受到影響
    其他少量缺值的部分如果是數值型的欄位使用平均數來處理，而非數值型的則是以出現次數最多的來補上

與上課內容關聯性 :
    這個競賽的主題和課程教的範例使用的預測模型一樣是線性回歸，差別在於這個主題在對於房價預測的因素更多，基本原理是一樣的，但在每個因素對房價的關聯性上需要花更多時間處理